# Hands-on RL with Ray’s RLlib
## A beginner’s tutorial for working with multi-agent environments, models, and algorithms

<img src="images/pitfall.jpg" width=250> <img src="images/tesla.jpg" width=254> <img src="images/forklifts.jpg" width=169> <img src="images/robots.jpg" width=252> <img src="images/dota2.jpg" width=213>

### Overview
“Hands-on RL with Ray’s RLlib” is a beginners tutorial for working with reinforcement learning (RL) environments, models, and algorithms using Ray’s RLlib library. RLlib offers high scalability, a large list of algos to choose from (offline, model-based, model-free, etc..), support for TensorFlow and PyTorch, and a unified API for a variety of applications. This tutorial includes a brief introduction to provide an overview of concepts (e.g. why RL) before proceeding to RLlib (multi- and single-agent) environments, neural network models, hyperparameter tuning, debugging, student exercises, Q/A, and more. All code will be provided as .py files in a GitHub repo.

### Intended Audience
* Python programmers who want to get started with reinforcement learning and RLlib.

### Prerequisites
* Some Python programming experience.
* Some familiarity with machine learning.
* *Helpful, but not required:* Experience in reinforcement learning and Ray.
* *Helpful, but not required:* Experience with TensorFlow or PyTorch.

### Requirements/Dependencies

To get this very notebook up and running on your local machine, you can follow these steps here:

Install conda (https://www.anaconda.com/products/individual)

Then ...

#### Quick `conda` setup instructions (Linux):
```
$ conda create -n rllib python=3.8
$ conda activate rllib
$ pip install ray[rllib]
$ pip install tensorflow  # <- either one works!
$ pip install torch  # <- either one works!
$ pip install jupyterlab
```

#### Quick `conda` setup instructions (Mac):
```
$ conda create -n rllib python=3.8
$ conda activate rllib
$ pip install cmake "ray[rllib]"
$ pip install tensorflow  # <- either one works!
$ pip install torch  # <- either one works!
$ pip install jupyterlab
```

#### Quick `conda` setup instructions (Win10):
```
$ conda create -n rllib python=3.8
$ conda activate rllib
$ pip install ray[rllib]
$ pip install [tensorflow|torch]  # <- either one works!
$ pip install jupyterlab
$ conda install pywin32
```

Also, for Win10 Atari support, we have to install atari_py from a different source (gym does not support Atari envs on Windows).

```
$ pip install git+https://github.com/Kojoley/atari-py.git
```

### Opening these tutorial files:
```
$ git clone https://github.com/sven1977/rllib_tutorials
$ cd rllib_tutorials
$ jupyter-lab
```

### Key Takeaways
* What is reinforcement learning and why RLlib?
* Core concepts of RLlib: Environments, Trainers, Policies, and Models.
* How to configure, hyperparameter-tune, and parallelize RLlib.
* RLlib debugging best practices.

### Tutorial Outline
1. RL and RLlib in a nutshell.
1. Defining an RL-solvable problem: Our first environment.
1. **Exercise No.1**: Environment loop.

(15min break)

1. Picking an algorithm and training our first RLlib Trainer.
1. Configurations and hyperparameters - Easy tuning with Ray Tune.
1. Fixing our experiment's config - Going multi-agent.
1. The "infinite laptop": Quick intro into how to use RLlib with Anyscale's product.
1. **Exercise No.2**: Run your own Ray RLlib+Tune experiment)
1. Neural network models - Provide your custom models using tf.keras or torch.nn.

(15min break)

1. Deeper dive into RLlib's parallelization architecture.
1. Specifying different compute resources and parallelization options through our config.
1. "Hacking in": Using callbacks to customize the RL loop and generate our own metrics.
1. **Exercise No.3**: Write your own custom callback.
1. "Hacking in (part II)" - Debugging with RLlib and PyCharm.
1. Checking on the "infinite laptop" - Did RLlib learn to solve the problem?

### Other Recommended Readings
* [Reinforcement Learning with RLlib in the Unity Game Engine](https://medium.com/distributed-computing-with-ray/reinforcement-learning-with-rllib-in-the-unity-game-engine-1a98080a7c0d)

<img src="images/unity3d_blog_post.png" width=400>

* [Attention Nets and More with RLlib's Trajectory View API](https://medium.com/distributed-computing-with-ray/attention-nets-and-more-with-rllibs-trajectory-view-api-d326339a6e65)
* [Intro to RLlib: Example Environments](https://medium.com/distributed-computing-with-ray/intro-to-rllib-example-environments-3a113f532c70)


## The RL cycle

<img src="images/rl-cycle.png" width=800>

### Coding/defining our "problem" via an RL environment.

We will use the following (adversarial) multi-agent environment
throughout this tutorial to demonstrate a large fraction of RLlib's
APIs, features, and customization options.

<img src="images/environment.png" width=800>

### A word or two on Spaces:

Spaces are used in ML to describe what possible/valid values inputs and outputs of a neural network can have.

RL environments also use them to describe what their valid observations and actions are.

Spaces are usually defined by their shape (e.g. 84x84x3 RGB images) and datatype (e.g. uint8 for RGB values between 0 and 255).
However, spaces could also be composed of other spaces (see Tuple or Dict spaces) or could be simply discrete with n fixed possible values
(represented by integers). For example, in our game, where each agent can only go up/down/left/right, the action space would be `Discrete(4)`
(no datatype, no shape needs to be defined here). Our observation space will be `MultiDiscrete([n, m])`, where n is the position of the agent observing and m is the position of the opposing agent, so if agent1 starts in the upper left corner and agent2 starts in the bottom right corner, agent1's observation would be: `[0, 63]` (in an 8 x 8 grid) and agent2's observation would be `[63, 0]`.

<img src="images/spaces.png" width=800>

In [1]:
# Let's code our multi-agent environment.

import gym
from gym.spaces import Discrete, MultiDiscrete
import numpy as np
import random

from ray.rllib.env.multi_agent_env import MultiAgentEnv


class MultiAgentArena(MultiAgentEnv):
    def __init__(self, config=None):
        config = config or {}
        # Dimensions of the grid.
        self.width = config.get("width", 10)
        self.height = config.get("height", 10)

        # End an episode after this many timesteps.
        self.timestep_limit = config.get("ts", 100)

        self.observation_space = MultiDiscrete([self.width * self.height,
                                                self.width * self.height])
        # 0=up, 1=right, 2=down, 3=left.
        self.action_space = Discrete(4)

        # Reset env.
        self.reset()
        
    def reset(self):
        """Returns initial observation of next(!) episode."""
        # Row-major coords.
        self.agent1_pos = [0, 0]  # upper left corner
        self.agent2_pos = [self.height - 1, self.width - 1]  # lower bottom corner

        # Accumulated rewards in this episode.
        self.agent1_R = 0.0
        self.agent2_R = 0.0

        # Reset agent1's visited fields.
        self.agent1_visited_fields = set([tuple(self.agent1_pos)])

        # How many timesteps have we done in this episode.
        self.timesteps = 0

        # Return the initial observation in the new episode.
        return self._get_obs()

    def step(self, action: dict):
        """
        Returns (next observation, rewards, dones, infos) after having taken the given actions.
        
        e.g.
        `action={"agent1": action_for_agent1, "agent2": action_for_agent2}`
        """
        
        # increase our time steps counter by 1.
        self.timesteps += 1
        # An episode is "done" when we reach the time step limit.
        is_done = self.timesteps >= self.timestep_limit

        # Agent2 always moves first.
        # events = [collision|agent1_new_field]
        events = self._move(self.agent2_pos, action["agent2"], is_agent1=False)
        events |= self._move(self.agent1_pos, action["agent1"], is_agent1=True)

        # Useful for rendering.
        self.collision = "collision" in events
            
        # Get observations (based on new agent positions).
        obs = self._get_obs()

        # Determine rewards based on the collected events:
        r1 = -1.0 if "collision" in events else 1.0 if "agent1_new_field" in events else -0.5
        r2 = 1.0 if "collision" in events else -0.1

        self.agent1_R += r1
        self.agent2_R += r2
        
        rewards = {
            "agent1": r1,
            "agent2": r2,
        }

        # Generate a `done` dict (per-agent and total).
        dones = {
            "agent1": is_done,
            "agent2": is_done,
            # special `__all__` key indicates that the episode is done for all agents.
            "__all__": is_done,
        }

        return obs, rewards, dones, {}  # <- info dict (not needed here).

    def _get_obs(self):
        """
        Returns obs dict (agent name to discrete-pos tuple) using each
        agent's current x/y-positions.
        """
        ag1_discrete_pos = self.agent1_pos[0] * self.width + \
            (self.agent1_pos[1] % self.width)
        ag2_discrete_pos = self.agent2_pos[0] * self.width + \
            (self.agent2_pos[1] % self.width)
        return {
            "agent1": np.array([ag1_discrete_pos, ag2_discrete_pos]),
            "agent2": np.array([ag2_discrete_pos, ag1_discrete_pos]),
        }

    def _move(self, coords, action, is_agent1):
        """
        Moves an agent (agent1 iff is_agent1=True, else agent2) from `coords` (x/y) using the
        given action (0=up, 1=right, etc..) and returns a resulting events dict:
        Agent1: "new" when entering a new field. "bumped" when having been bumped into by agent2.
        Agent2: "bumped" when bumping into agent1 (agent1 then gets -1.0).
        """
        orig_coords = coords[:]
        # Change the row: 0=up (-1), 2=down (+1)
        coords[0] += -1 if action == 0 else 1 if action == 2 else 0
        # Change the column: 1=right (+1), 3=left (-1)
        coords[1] += 1 if action == 1 else -1 if action == 3 else 0

        # Solve collisions.
        # Make sure, we don't end up on the other agent's position.
        # If yes, don't move (we are blocked).
        if (is_agent1 and coords == self.agent2_pos) or (not is_agent1 and coords == self.agent1_pos):
            coords[0], coords[1] = orig_coords
            # Agent2 blocked agent1 (agent1 tried to run into agent2)
            # OR Agent2 bumped into agent1 (agent2 tried to run into agent1)
            return {"collision"}

        # No agent blocking -> check walls.
        if coords[0] < 0:
            coords[0] = 0
        elif coords[0] >= self.height:
            coords[0] = self.height - 1
        if coords[1] < 0:
            coords[1] = 0
        elif coords[1] >= self.width:
            coords[1] = self.width - 1

        # If agent1 -> "new" if new tile covered.
        if is_agent1 and not tuple(coords) in self.agent1_visited_fields:
            self.agent1_visited_fields.add(tuple(coords))
            return {"agent1_new_field"}
        # No new tile for agent1.
        return set()

    def render(self, mode=None):
        print("_" * (self.width + 2))
        for r in range(self.height):
            print("|", end="")
            for c in range(self.width):
                field = r * self.width + c % self.width
                if self.agent1_pos == [r, c]:
                    print("1", end="")
                elif self.agent2_pos == [r, c]:
                    print("2", end="")
                elif (r, c) in self.agent1_visited_fields:
                    print(".", end="")
                else:
                    print(" ", end="")
            print("|")
        print("‾" * (self.width + 2))
        print(f"{'!!Collision!!' if self.collision else ''}")
        print("R1={: .1f}".format(self.agent1_R))
        print("R2={: .1f}".format(self.agent2_R))
        print()


env = MultiAgentArena()

obs = env.reset()

# Agent1 will move down, Agent2 moves up.
obs, rewards, dones, infos = env.step(action={"agent1": 2, "agent2": 0})

env.render()

print("Agent1's x/y position={}".format(env.agent1_pos))
print("Agent2's x/y position={}".format(env.agent2_pos))
print("Env timesteps={}".format(env.timesteps))


/Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


____________
|.         |
|1         |
|          |
|          |
|          |
|          |
|          |
|          |
|         2|
|          |
‾‾‾‾‾‾‾‾‾‾‾‾

R1= 1.0
R2=-0.1

Agent1's x/y position=[1, 0]
Agent2's x/y position=[8, 9]
Env timesteps=1


## Exercise No 1

<hr />

<img src="images/exercise1.png" width=400>

In the cell above, we performed a `reset()` and a single `step()` call. To walk through an entire episode, one would normally call `step()` repeatedly (with different actions) until the returned `done` dict has the "agent1" or "agent2" (or "__all__") key set to True. Your task is to write an "environment loop" that runs for exactly one episode using our `MultiAgentArena` class.

Follow these instructions here to get this done.

1. Create an env object.
1. `reset` your environment to get the first (initial) observation.
1. Compute the actions for "agent1" and "agent2" calling `DummyTrainer.compute_action([obs])` twice and putting the results into an action dict to be passed into `step()`, just like it's done in the above cell (where we do a single `step()`).
1. Repeat this, `step`ing through an entire episode.
1. When an episode is done, `step()` will return a done dict with key `__all__` set to True.
1. If you feel, this is way too easy for you ;) , try to extract each agent's reward, sum it up over the episode and - at the end of the episode - print out each agent's accumulated reward (also called the "return" of an episode).

**Good luck! :)**


In [22]:
class DummyTrainer:
    """Dummy Trainer class used in Exercise #1.

    Use its `compute_action` method to get a new action for one of the agents,
    given the agent's observation (a single discrete value encoding the field
    the agent is currently in).
    """

    def compute_action(self, single_agent_obs=None):
        # Returns a random action for a single agent.
        return np.random.randint(4)  # Discrete(4) -> return rand int between 0 and 3 (incl. 3).

dummy_trainer = DummyTrainer()
# Check, whether it's working.
for _ in range(3):
    # Get action for agent1 (providing agent1's and agent2's positions).
    print("action_agent1={}".format(dummy_trainer.compute_action(np.array([0, 99]))))

    # Get action for agent2 (providing agent2's and agent1's positions).
    print("action_agent2={}".format(dummy_trainer.compute_action(np.array([99, 0]))))

    print()

action_agent1=3
action_agent2=1

action_agent1=0
action_agent2=1

action_agent1=0
action_agent2=2



Write your solution code into this cell here:

In [23]:
# !LIVE CODING!

# Leave the following as-is. It'll help us with rendering the env in this very cell's output.
import time
from ipywidgets import Output
from IPython import display
import time

out = Output()
display.display(out)

with out:

    # Solution to Exercise #1:
    # Start coding here inside this `with`-block:
    # 1) Reset the env.
    obs = env.reset()  # start new episode

    # 2) Enter an infinite while loop (to step through the episode).
    while env.timesteps < 100:
        # 3) Calculate both agents' actions individually, using dummy_trainer.compute_action([individual agent's obs])
        a1 = dummy_trainer.compute_action(obs["agent1"])
        a2 = dummy_trainer.compute_action(obs["agent2"])

        # 4) Compile the actions dict from both individual actions.
        actions = {
            "agent1": a1, "agent2": a2,
        }

        # 5) Send the actions dict to the env's `step()` method to receive: obs, rewards, dones, info dicts
        obs, rewards, dones, _ = env.step(actions)

        # 6) We'll do this together: Render the env.
        # Don't write any code here (skip directly to 7).
        out.clear_output(wait=True)
        time.sleep(0.05)
        env.render()

        # 7) Check, whether the episde is done, if yes, break out of the while loop.
        if dones["agent1"] is True:
            break

# 8) Run it! :)

Output()

------------------
## 15 min break :)
------------------

### And now for something completely different:
#### Plugging in RLlib!

In [24]:
import numpy as np
import pprint
import ray

# Start a new instance of Ray (when running this tutorial locally) or
# connect to an already running one (when running this tutorial through Anyscale).

ray.init()  # Hear the engine humming? ;)

# In case you encounter the following error during our tutorial: `RuntimeError: Maybe you called ray.init twice by accident?`
# Try: `ray.shutdown() + ray.init()` or `ray.init(ignore_reinit_error=True)`

2021-06-24 10:42:20,283	INFO services.py:1272 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.179',
 'raylet_ip_address': '192.168.0.179',
 'redis_address': '192.168.0.179:6379',
 'object_store_address': '/tmp/ray/session_2021-06-24_10-42-18_600631_2196/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-06-24_10-42-18_600631_2196/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-06-24_10-42-18_600631_2196',
 'metrics_export_port': 63988,
 'node_id': 'fb379f8d24eba01da8c8027447994ca7c9754701cb243183c55e28d7'}

### Picking an RLlib algorithm - We'll use PPO throughout this tutorial (one-size-fits-all-kind-of-algo)

<img src="images/rllib_algos.png" width=800>

https://docs.ray.io/en/master/rllib-algorithms.html#available-algorithms-overview

In [25]:
# Import a Trainable (one of RLlib's built-in algorithms):
# We use the PPO algorithm here b/c its very flexible wrt its supported
# action spaces and model types and b/c it learns well almost any problem.
from ray.rllib.agents.ppo import PPOTrainer

# Specify a very simple config, defining our environment and some environment
# options (see environment.py).
config = {
    "env": MultiAgentArena,  # "my_env" <- if we previously have registered the env with `tune.register_env("[name]", lambda config: [returns env object])`.
    "env_config": {
        "config": {
            "width": 10,
            "height": 10,
            "ts": 100,
        },
    },

    # !PyTorch users!
    #"framework": "torch",  # If users have chosen to install torch instead of tf.

    "create_env_on_driver": True,
}
# Instantiate the Trainer object using above config.
rllib_trainer = PPOTrainer(config=config)
rllib_trainer

2021-06-24 10:42:23,897	INFO trainer.py:671 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-06-24 10:42:23,898	INFO trainer.py:696 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-06-24 10:42:33,238	WARNING util.py:53 -- Install gputil for GPU system monitoring.


PPO

### Ready to train with RLlib's PPO algorithm

That's it, we are ready to train.
Calling `Trainer.train()` will execute a single "training iteration".

One iteration for most algos involves:

1) sampling from the environment(s)
2) using the sampled data (observations, actions taken, rewards) to update the policy model (neural network), such that it would pick better actions in the future, leading to higher rewards.

Let's try it out:

In [26]:
results = rllib_trainer.train()

# Delete the config from the results for clarity.
# Only the stats will remain, then.
del results["config"]
# Pretty print the stats.
pprint.pprint(results)

{'agent_timesteps_total': 4000,
 'custom_metrics': {},
 'date': '2021-06-24_10-42-36',
 'done': False,
 'episode_len_mean': 100.0,
 'episode_media': {},
 'episode_reward_max': 14.999999999999998,
 'episode_reward_mean': -9.104999999999999,
 'episode_reward_min': -34.50000000000005,
 'episodes_this_iter': 20,
 'episodes_total': 20,
 'experiment_id': '74285469b30e44cdbdfb8f3ba86dfb6c',
 'hist_stats': {'episode_lengths': [100,
                                    100,
                                    100,
                                    100,
                                    100,
                                    100,
                                    100,
                                    100,
                                    100,
                                    100,
                                    100,
                                    100,
                                    100,
                                    100,
                                    100

### Going from single policy (RLlib's default) to multi-policy:

So far, our experiment has been ill-configured, because both
agents, which should behave differently due to their different
tasks and reward functions, learn the same policy: the "default_policy",
which RLlib always provides if you don't configure anything else.
Remember that RLlib does not know at Trainer setup time, how many and which agents
the environment will "produce". Agent control (adding agents, removing them, terminating
episodes for agents) is entirely in the Env's hands.
Let's fix our single policy problem and introduce the "multiagent" API.

<img src="images/from_single_agent_to_multi_agent.png" width=800>

In order to turn on RLlib's multi-agent functionality, we need two things:

1. A policy mapping function, mapping agent IDs (e.g. a string like "agent1", produced by the environment in the returned observation/rewards/dones-dicts) to a policy ID (another string, e.g. "policy1", which is under our control).
1. A policies definition dict, mapping policy IDs (e.g. "policy1") to 4-tuples consisting of 1) policy class (None for using the default class), 2) observation space, 3) action space, and 4) config overrides (empty dict for no overrides and using the Trainer's main config dict).

Let's take a closer look:

In [27]:
# Define the policies definition dict:
# Each policy in there is defined by its ID (key) mapping to a 4-tuple (value):
# - Policy class (None for using the "default" class, e.g. PPOTFPolicy for PPO+tf or PPOTorchPolicy for PPO+torch).
# - obs-space (we get this directly from our already created env object).
# - act-space (we get this directly from our already created env object).
# - config-overrides dict (leave empty for using the Trainer's config as-is)
policies = {
    "policy1": (None, env.observation_space, env.action_space, {}),
    "policy2": (None, env.observation_space, env.action_space, {"lr": 0.0002}),
}
# Note that now we won't have a "default_policy" anymore, just "policy1" and "policy2".

# Define an agent->policy mapping function.
# Which agents (defined by the environment) use which policies (defined by us)?
# The mapping here is M (agents) -> N (policies), where M >= N.
def policy_mapping_fn(agent_id: str):
    # Make sure agent ID is valid.
    assert agent_id in ["agent1", "agent2"], f"ERROR: invalid agent ID {agent_id}!"
    # Map agent1 to policy1, and agent2 to policy2.
    return "policy1" if agent_id == "agent1" else "policy2"

# We could - if we wanted - specify, which policies should be learnt (by default, RLlib learns all).
# Non-learnt policies will be frozen and not updated:
# policies_to_train = ["policy1", "policy2"]

# Adding the above to our config.
config.update({
    "multiagent": {
        "policies": policies,
        "policy_mapping_fn": policy_mapping_fn,
        # We'll leave this empty: Means, we train both policy1 and policy2.
        # "policies_to_train": policies_to_train,
    },
})

pprint.pprint(config)
print()
print(f"agent1 is now mapped to {policy_mapping_fn('agent1')}")
print(f"agent2 is now mapped to {policy_mapping_fn('agent2')}")

{'create_env_on_driver': True,
 'env': <class '__main__.MultiAgentArena'>,
 'env_config': {'config': {'height': 10, 'ts': 100, 'width': 10}},
 'multiagent': {'policies': {'policy1': (None,
                                         MultiDiscrete([100 100]),
                                         Discrete(4),
                                         {}),
                             'policy2': (None,
                                         MultiDiscrete([100 100]),
                                         Discrete(4),
                                         {'lr': 0.0002})},
                'policy_mapping_fn': <function policy_mapping_fn at 0x7fbc3a8b2af0>}}

agent1 is now mapped to policy1
agent2 is now mapped to policy2


In [28]:
# Recreate our Trainer (we cannot just change the config on-the-fly).
rllib_trainer.stop()

# Using our updated (now multiagent!) config dict.
rllib_trainer = PPOTrainer(config=config)
rllib_trainer

2021-06-24 10:42:49,739	INFO trainable.py:101 -- Trainable.setup took 12.972 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2021-06-24 10:42:49,740	WARNING util.py:53 -- Install gputil for GPU system monitoring.


PPO

Now that we are setup correctly with two policies as per our "multiagent" config, let's call `train()` on the new Trainer several times (what about 10 times?).

In [29]:
# Run `train()` n times. Repeatedly call `train()` now to see rewards increase.
# Move on once you see (agent1 + agent2) episode rewards of 10.0 or more.
for _ in range(10):
    results = rllib_trainer.train()
    print(f"Iteration={rllib_trainer.iteration}: R(\"return\")={results['episode_reward_mean']}")

Iteration=1: R("return")=-8.587500000000002
Iteration=2: R("return")=-5.167499999999999
Iteration=3: R("return")=-4.460999999999993
Iteration=4: R("return")=-2.3789999999999885
Iteration=5: R("return")=-1.7069999999999876
Iteration=6: R("return")=-0.26699999999998636
Iteration=7: R("return")=1.4850000000000128
Iteration=8: R("return")=2.2110000000000114
Iteration=9: R("return")=3.3690000000000078
Iteration=10: R("return")=3.6810000000000063


In [30]:
# Do another loop, but this time, we will print out each policies' individual rewards.
for _ in range(10):
    results = rllib_trainer.train()
    r1 = results['policy_reward_mean']['policy1']
    r2 = results['policy_reward_mean']['policy2']
    r = r1 + r2
    print(f"Iteration={rllib_trainer.iteration}: R(\"return\")={r} R1={r1} R2={r2}")

Iteration=11: R1=11.675 R2=-6.655999999999988
Iteration=12: R1=13.175 R2=-6.325999999999986
Iteration=13: R1=12.185 R2=-5.929999999999989
Iteration=14: R1=12.515 R2=-5.995999999999988
Iteration=15: R1=12.13 R2=-5.610999999999991
Iteration=16: R1=13.305 R2=-5.3249999999999895
Iteration=17: R1=14.36 R2=-5.2699999999999925
Iteration=18: R1=15.555 R2=-5.78699999999999
Iteration=19: R1=16.79 R2=-5.698999999999991
Iteration=20: R1=17.795 R2=-5.66599999999999


#### !OPTIONAL HACK! (<-- we will not do these during the tutorial, but feel free to try these cells by yourself)

Use the above solution of Exercise #1 and replace our `dummy_trainer` in that solution
with the now trained `rllib_trainer`. You should see a better performance of the two agents.

However, keep in mind that we are mostly training agent1 as we only trian a single policy and agent1
is the "easier" one to collect high rewards with.

#### !OPTIONAL HACK!

Feel free to play around with the following code in order to learn how RLlib - under the hood - calculates actions from the environment's observations using Policies and their model(s) inside our Trainer object):

In [31]:
# Let's actually "look inside" our Trainer to see what's in there.
from ray.rllib.utils.numpy import softmax

# To get to one of the policies inside the Trainer, use `Trainer.get_policy([policy ID])`:
policy = rllib_trainer.get_policy("policy1")
print(f"Our (only!) Policy right now is: {policy}")

# To get to the model inside any policy, do:
model = policy.model
#print(f"Our Policy's model is: {model}")

# Print out the policy's action and observation spaces.
print(f"Our Policy's observation space is: {policy.observation_space}")
print(f"Our Policy's action space is: {policy.action_space}")

# Produce a random obervation (B=1; batch of size 1).
obs = np.array([policy.observation_space.sample()])
# Alternatively for PyTorch:
#import torch
#obs = torch.from_numpy(obs)

# Get the action logits (as tf tensor).
# If you are using torch, you would get a torch tensor here.
logits, _ = model({"obs": obs})
logits

# Numpyize the tensor by running `logits` through the Policy's own tf.Session.
logits_np = policy.get_session().run(logits)
# For torch, you can simply do: `logits_np = logits.detach().cpu().numpy()`.

# Convert logits into action probabilities and remove the B=1.
action_probs = np.squeeze(softmax(logits_np))

# Sample an action, using the probabilities.
action = np.random.choice([0, 1, 2, 3], p=action_probs)

# Print out the action.
print(f"sampled action={action}")

Our (only!) Policy right now is: <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7fbc409bd160>
Our Policy's observation space is: Box(-1.0, 1.0, (200,), float32)
Our Policy's action space is: Discrete(4)
sampled action=3


### Saving and restoring a trained Trainer.
Currently, `rllib_trainer` is in an already trained state.
It holds optimized weights in its Policy's model that allow it to act
already somewhat smart in our environment when given an observation.

However, if we closed this notebook right now, all the effort would have been for nothing.
Let's therefore save the state of our trainer to disk for later!

In [32]:
# We use the `Trainer.save()` method to create a checkpoint.
checkpoint_file = rllib_trainer.save()
print(f"Trainer (at iteration {rllib_trainer.iteration} was saved in '{checkpoint_file}'!")

# Here is what a checkpoint directory contains:
print("The checkpoint directory contains the following files:")
import os
os.listdir(os.path.dirname(checkpoint_file))

Trainer (at iteration 20 was saved in '/Users/sven/ray_results/PPO_MultiAgentArena_2021-06-24_10-42-36tjwkrob9/checkpoint_000020/checkpoint-20'!
The checkpoint directory contains the following files:


['checkpoint-20', 'checkpoint-20.tune_metadata', '.is_checkpoint']

### Restoring and evaluating a Trainer
In the following cell, we'll learn how to restore a saved Trainer from a checkpoint file.

We'll also evaluate a completely new Trainer (should act more or less randomly) vs an already trained one (the one we just restored from the created checkpoint file).

In [33]:
# Pretend, we wanted to pick up training from a previous run:
new_trainer = PPOTrainer(config=config)
# Evaluate the new trainer (this should yield random results).
results = new_trainer.evaluate()
print(f"Evaluating new trainer: R={results['evaluation']['episode_reward_mean']}")

# Restoring the trained state into the `new_trainer` object.
print(f"Before restoring: Trainer is at iteration={new_trainer.iteration}")
new_trainer.restore(checkpoint_file)
print(f"After restoring: Trainer is at iteration={new_trainer.iteration}")

# Evaluate again (this should yield results we saw after having trained our saved agent).
results = new_trainer.evaluate()
print(f"Evaluating restored trainer: R={results['evaluation']['episode_reward_mean']}")

2021-06-24 10:45:16,711	INFO trainable.py:101 -- Trainable.setup took 13.248 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2021-06-24 10:45:16,713	WARNING util.py:53 -- Install gputil for GPU system monitoring.


Evaluating new trainer: R=-7.379999999999993
Before restoring: Trainer is at iteration=0


2021-06-24 10:45:19,816	INFO trainable.py:377 -- Restored on 192.168.0.179 from checkpoint: /Users/sven/ray_results/PPO_MultiAgentArena_2021-06-24_10-42-36tjwkrob9/checkpoint_000020/checkpoint-20
2021-06-24 10:45:19,817	INFO trainable.py:385 -- Current state after restoring: {'_iteration': 20, '_timesteps_total': None, '_time_total': 132.94485211372375, '_episodes_total': 800}


After restoring: Trainer is at iteration=20
Evaluating restored trainer: R=15.284999999999972


In order to release all resources from a Trainer, you can use a Trainer's `stop()` method.
You should definitley run this cell as it frees resources that we'll need later in this tutorial, when we'll do parallel hyperparameter sweeps.

In [34]:
rllib_trainer.stop()
new_trainer.stop()

### Moving stuff to the professional level: RLlib in connection w/ Ray Tune

Running any experiments through Ray Tune is the recommended way of doing things with RLlib. If you look at our
<a href="https://github.com/ray-project/ray/tree/master/rllib/examples">examples scripts folder</a>, you will see that almost all of the scripts use Ray Tune to run the particular RLlib workload demonstrated in each script.

<img src="images/rllib_and_tune.png" width=400>

When setting up hyperparameter sweeps for Tune, we'll do this in our already familiar config dict.

So let's take a quick look at our PPO algo's default config to understand, which hyperparameters we may want to play around with:

In [35]:
# Configuration dicts and Ray Tune.
# Where are the default configuration dicts stored?

# PPO algorithm:
from ray.rllib.agents.ppo import DEFAULT_CONFIG as PPO_DEFAULT_CONFIG
print(f"PPO's default config is:")
pprint.pprint(PPO_DEFAULT_CONFIG)

# DQN algorithm:
#from ray.rllib.agents.dqn import DEFAULT_CONFIG as DQN_DEFAULT_CONFIG
#print(f"DQN's default config is:")
#pprint.pprint(DQN_DEFAULT_CONFIG)

# Common (all algorithms).
#from ray.rllib.agents.trainer import COMMON_CONFIG
#print(f"RLlib Trainer's default config is:")
#pprint.pprint(COMMON_CONFIG)

PPO's default config is:
{'_fake_gpus': False,
 'batch_mode': 'truncate_episodes',
 'callbacks': <class 'ray.rllib.agents.callbacks.DefaultCallbacks'>,
 'clip_actions': True,
 'clip_param': 0.3,
 'clip_rewards': None,
 'collect_metrics_timeout': 180,
 'compress_observations': False,
 'create_env_on_driver': False,
 'custom_eval_function': None,
 'custom_resources_per_worker': {},
 'eager_tracing': False,
 'entropy_coeff': 0.0,
 'entropy_coeff_schedule': None,
 'env': None,
 'env_config': {},
 'env_task_fn': None,
 'evaluation_config': {},
 'evaluation_interval': None,
 'evaluation_num_episodes': 10,
 'evaluation_num_workers': 0,
 'evaluation_parallel_to_training': False,
 'exploration_config': {'type': 'StochasticSampling'},
 'explore': True,
 'extra_python_environs_for_driver': {},
 'extra_python_environs_for_worker': {},
 'fake_sampler': False,
 'framework': 'tf',
 'gamma': 0.99,
 'grad_clip': None,
 'horizon': None,
 'ignore_worker_failures': False,
 'in_evaluation': False,
 'input'

### Let's do a very simple grid-search over two learning rates with tune.run().

In particular, we will try the learning rates 0.00005 and 0.5 using `tune.grid_search([...])`
inside our config dict:

In [36]:
# Plugging in Ray Tune.
# Note that this is the recommended way to run any experiments with RLlib.
# Reasons:
# - Tune allows you to do hyperparameter tuning in a user-friendly way
#   and at large scale!
# - Tune automatically allocates needed resources for the different
#   hyperparam trials and experiment runs on a cluster.

from ray import tune

# Running stuff with tune, we can re-use the exact
# same config that we used when working with RLlib directly!
tune_config = config.copy()

# Let's add our first hyperparameter search via our config.
# How about we try two different learning rates? Let's say 0.00005 and 0.5 (ouch!).
tune_config["lr"] = tune.grid_search([0.00005, 0.5])  # <- 0.5? again: ouch!
tune_config["train_batch_size"] = tune.grid_search([3000, 4000])

# Now that we will run things "automatically" through tune, we have to
# define one or more stopping criteria.
# Tune will stop the run, once any single one of the criteria is matched (not all of them!).
stop = {
    # Note that the keys used here can be anything present in the above `rllib_trainer.train()` output dict.
    "training_iteration": 5,
    "episode_reward_mean": 20.0,
}

# "PPO" is a registered name that points to RLlib's PPOTrainer.
# See `ray/rllib/agents/registry.py`

# Run a simple experiment until one of the stopping criteria is met.
tune.run(
    "PPO",
    config=tune_config,
    stop=stop,

    # Note that no trainers will be returned from this call here.
    # Tune will create n Trainers internally, run them in parallel and destroy them at the end.
    # However, you can ...
    checkpoint_at_end=True,  # ... create a checkpoint when done.
    checkpoint_freq=10,  # ... create a checkpoint every 10 training iterations.
)

Trial name,status,loc,lr,train_batch_size
PPO_MultiAgentArena_83919_00000,PENDING,,5e-05,3000
PPO_MultiAgentArena_83919_00001,PENDING,,0.5,3000
PPO_MultiAgentArena_83919_00002,PENDING,,5e-05,4000
PPO_MultiAgentArena_83919_00003,PENDING,,0.5,4000


(pid=2675) 2021-06-24 10:45:29,348	INFO trainer.py:671 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=2675) 2021-06-24 10:45:29,348	INFO trainer.py:696 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=2675) 2021-06-24 10:45:29,349	WARNING ppo.py:135 -- `train_batch_size` (3000) cannot be achieved with your other settings (num_workers=2 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 1500.
(pid=2677) 2021-06-24 10:45:29,348	INFO trainer.py:671 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=2677) 2021-06-24 10:45:29,348	INFO trainer.py:696 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=2670) 2021-06-24 10:45:29,348	INFO trainer.py:671 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(p

Result for PPO_MultiAgentArena_83919_00000:
  agent_timesteps_total: 6000
  custom_metrics: {}
  date: 2021-06-24_10-45-54
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 15.000000000000014
  episode_reward_mean: -8.909999999999993
  episode_reward_min: -34.500000000000036
  episodes_this_iter: 30
  episodes_total: 30
  experiment_id: 43c5f1bf54a446d6ac106c951f38d022
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      policy1:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 4.999999873689376e-05
          entropy: 1.3664278984069824
          entropy_coeff: 0.0
          kl: 0.02037949115037918
          model: {}
          policy_loss: -0.05178745463490486
          total_loss: 53.435001373291016
          vf_explained_var: 0.13389408588409424
          vf_loss: 53.48271560668945
      policy2:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 0.00019999999494757503
   

Trial name,status,loc,lr,train_batch_size,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_83919_00000,RUNNING,192.168.0.179:2675,5e-05,3000,1,10.0594,3000,-8.91,15,-34.5,100
PPO_MultiAgentArena_83919_00001,RUNNING,,0.5,3000,,,,,,,
PPO_MultiAgentArena_83919_00002,RUNNING,,5e-05,4000,,,,,,,
PPO_MultiAgentArena_83919_00003,RUNNING,,0.5,4000,,,,,,,


Result for PPO_MultiAgentArena_83919_00001:
  agent_timesteps_total: 6000
  custom_metrics: {}
  date: 2021-06-24_10-45-54
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 15.000000000000016
  episode_reward_mean: -5.909999999999995
  episode_reward_min: -34.50000000000003
  episodes_this_iter: 30
  episodes_total: 30
  experiment_id: 38bdf32c865b44ec9dfdfa051cfd3bca
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      policy1:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 0.5
          entropy: 0.078101746737957
          entropy_coeff: 0.0
          kl: 18.170473098754883
          model: {}
          policy_loss: 0.47826075553894043
          total_loss: 56.070186614990234
          vf_explained_var: 0.009776771068572998
          vf_loss: 51.957828521728516
      policy2:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 0.00019999999494757503
          entropy: 1.34

...

Result for PPO_MultiAgentArena_83919_00001:
  agent_timesteps_total: 30000
  custom_metrics: {}
  date: 2021-06-24_10-46-40
  done: true
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: -15.299999999999994
  episode_reward_mean: -43.872000000000064
  episode_reward_min: -48.00000000000008
  episodes_this_iter: 30
  episodes_total: 150
  experiment_id: 38bdf32c865b44ec9dfdfa051cfd3bca
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      policy1:
        learner_stats:
          cur_kl_coeff: 1.0125000476837158
          cur_lr: 0.5
          entropy: 0.004095461219549179
          entropy_coeff: 0.0
          kl: 0.16092397272586823
          model: {}
          policy_loss: 0.051739536225795746
          total_loss: 95.20844268798828
          vf_explained_var: 0.10867179930210114
          vf_loss: 94.99378204345703
      policy2:
        learner_stats:
          cur_kl_coeff: 0.675000011920929
          cur_lr: 0.00019999999494757503
          entropy: 1.

Trial name,status,loc,lr,train_batch_size,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_83919_00002,RUNNING,192.168.0.179:2674,5e-05,4000,5,68.7927,20000,1.743,27.6,-16.5,100
PPO_MultiAgentArena_83919_00003,RUNNING,192.168.0.179:2677,0.5,4000,4,58.8085,16000,-41.406,-25.5,-46.5,100
PPO_MultiAgentArena_83919_00000,TERMINATED,,5e-05,3000,5,54.9213,15000,1.089,23.1,-24,100
PPO_MultiAgentArena_83919_00001,TERMINATED,,0.5,3000,5,55.0482,15000,-43.872,-15.3,-48,100


Result for PPO_MultiAgentArena_83919_00003:
  agent_timesteps_total: 40000
  custom_metrics: {}
  date: 2021-06-24_10-46-53
  done: true
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: -21.900000000000006
  episode_reward_mean: -36.06600000000004
  episode_reward_min: -46.500000000000064
  episodes_this_iter: 40
  episodes_total: 200
  experiment_id: 777bea0709a34c368f94f4b08cbe8c9b
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      policy1:
        learner_stats:
          cur_kl_coeff: 0.11249999701976776
          cur_lr: 0.5
          entropy: 0.0
          entropy_coeff: 0.0
          kl: 0.0
          model: {}
          policy_loss: -0.0009185558883473277
          total_loss: 146.60562133789062
          vf_explained_var: 0.07484202086925507
          vf_loss: 146.60653686523438
      policy2:
        learner_stats:
          cur_kl_coeff: 1.0125000476837158
          cur_lr: 0.00019999999494757503
          entropy: 1.092926025390625
          e

Trial name,status,loc,lr,train_batch_size,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_83919_00000,TERMINATED,,5e-05,3000,5,54.9213,15000,1.089,23.1,-24,100
PPO_MultiAgentArena_83919_00001,TERMINATED,,0.5,3000,5,55.0482,15000,-43.872,-15.3,-48,100
PPO_MultiAgentArena_83919_00002,TERMINATED,,5e-05,4000,5,68.7927,20000,1.743,27.6,-16.5,100
PPO_MultiAgentArena_83919_00003,TERMINATED,,0.5,4000,5,68.7797,20000,-36.066,-21.9,-46.5,100


2021-06-24 10:46:54,581	INFO tune.py:549 -- Total run time: 91.50 seconds (90.98 seconds for the tuning loop).


### Why did we use 6 CPUs in the tune run above (3 CPUs per trial)?

PPO - by default - uses 2 "rollout" workers (`num_workers=2`). These are Ray Actors that have their own environment copy(ies) and step through those in parallel. On top of these two "rollout" workers, every Trainer in RLlib always also has a "local" worker, which - in case of PPO - handles the learning updates. This gives us 3 workers (2 rollout + 1 local learner), which require 3 CPUs.

## Exercise No 2

<hr />

Using the `tune_config` that we have built so far, let's run another `tune.run()`, but apply the following changes to our setup this time:
- Setup only 1 learning rate under the "lr" config key. Chose the (seemingly) best value from the run in the previous cell (the one that yielded the highest avg. reward).
- Setup only 1 train batch size under the "train_batch_size" config key. Chose the (seemingly) best value from the run in the previous cell (the one that yielded the highest avg. reward).
- Set `num_workers` to 5, which will allow us to run more environment "rollouts" in parallel and to collect training batches more quickly.
- Set the `num_envs_per_worker` config parameter to 5. This will clone our env on each rollout worker, and thus parallelize action computing forward passes through our neural networks.

Other than that, use the exact same args as in our `tune.run()` call in the previous cell.

**Good luck! :)**


In [37]:
# !LIVE CODING!

# Solution to Exercise #2

# Run for longer this time (100 iterations) and try to reach 40.0 reward (sum of both agents).
stop = {
    "training_iteration": 180,  # we have the 15min break now to run this many iterations
    "episode_reward_mean": 60.0,  # sum of both agents' rewards. Probably won't reach it, but we should try nevertheless :)
}

# tune_config.update({
# ???
# })

# analysis = tune.run(...)

Trial name,status,loc
PPO_MultiAgentArena_62cb1_00000,PENDING,


(pid=2801) 2021-06-24 10:51:43,603	INFO trainer.py:671 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=2801) 2021-06-24 10:51:43,604	INFO trainer.py:696 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=2801) 2021-06-24 10:51:43,604	WARNING ppo.py:135 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=5 num_envs_per_worker=5 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 160.
(raylet) /Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   warnings.warn(
(raylet) /Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.8/site-packages/ray/autoscaler/_p

Result for PPO_MultiAgentArena_62cb1_00000:
  agent_timesteps_total: 8000
  custom_metrics: {}
  date: 2021-06-24_10-52-02
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 17.999999999999936
  episode_reward_mean: -8.483999999999995
  episode_reward_min: -34.500000000000036
  episodes_this_iter: 25
  episodes_total: 25
  experiment_id: 3d6fd9cc179a437092430c6672b67fdc
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      policy1:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 9.999999747378752e-05
          entropy: 1.3588711023330688
          entropy_coeff: 0.0
          kl: 0.027996812015771866
          model: {}
          policy_loss: -0.05669461190700531
          total_loss: 44.654815673828125
          vf_explained_var: 0.09232431650161743
          vf_loss: 44.70591354370117
      policy2:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 0.00019999999494757503
  

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_62cb1_00000,RUNNING,192.168.0.179:2801,1,5.33662,4000,-8.484,18,-34.5,100


Result for PPO_MultiAgentArena_62cb1_00000:
  agent_timesteps_total: 24000
  custom_metrics: {}
  date: 2021-06-24_10-52-11
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 22.499999999999943
  episode_reward_mean: -5.075999999999994
  episode_reward_min: -37.50000000000004
  episodes_this_iter: 25
  episodes_total: 100
  experiment_id: 3d6fd9cc179a437092430c6672b67fdc
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      policy1:
        learner_stats:
          cur_kl_coeff: 0.44999998807907104
          cur_lr: 9.999999747378752e-05
          entropy: 1.2776226997375488
          entropy_coeff: 0.0
          kl: 0.026900572702288628
          model: {}
          policy_loss: -0.07335038483142853
          total_loss: 29.200061798095703
          vf_explained_var: 0.20351383090019226
          vf_loss: 29.261306762695312
      policy2:
        learner_stats:
          cur_kl_coeff: 0.44999998807907104
          cur_lr: 0.00019999999494757503


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_62cb1_00000,RUNNING,192.168.0.179:2801,198,928.169,792000,38.325,57.3,0.3,100


Result for PPO_MultiAgentArena_62cb1_00000:
  agent_timesteps_total: 1592000
  custom_metrics: {}
  date: 2021-06-24_11-07-36
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 57.5999999999999
  episode_reward_mean: 38.35499999999991
  episode_reward_min: 0.3000000000000218
  episodes_this_iter: 50
  episodes_total: 7950
  experiment_id: 3d6fd9cc179a437092430c6672b67fdc
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      policy1:
        learner_stats:
          cur_kl_coeff: 0.5062500238418579
          cur_lr: 9.999999747378752e-05
          entropy: 0.20364879071712494
          entropy_coeff: 0.0
          kl: 0.011059199459850788
          model: {}
          policy_loss: -0.022203342989087105
          total_loss: 34.86799621582031
          vf_explained_var: 0.7124600410461426
          vf_loss: 34.88459777832031
      policy2:
        learner_stats:
          cur_kl_coeff: 0.854296863079071
          cur_lr: 0.00019999999494757503
    

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_62cb1_00000,RUNNING,192.168.0.179:2801,200,937.611,800000,39.219,57.6,15.6,100


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_62cb1_00000,TERMINATED,,200,937.611,800000,39.219,57.6,15.6,100


(pid=2836) 2021-06-24 11:07:41,997	ERROR worker.py:409 -- SystemExit was raised from the worker
(pid=2836) Traceback (most recent call last):
(pid=2836)   File "python/ray/_raylet.pyx", line 497, in ray._raylet.execute_task
(pid=2836)   File "python/ray/_raylet.pyx", line 501, in ray._raylet.execute_task
(pid=2836)   File "python/ray/_raylet.pyx", line 451, in ray._raylet.execute_task.function_executor
(pid=2836)   File "/Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.8/site-packages/ray/_private/function_manager.py", line 563, in actor_method_executor
(pid=2836)     return method(__ray_actor, *args, **kwargs)
(pid=2836)   File "/Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.8/site-packages/ray/actor.py", line 1015, in __ray_terminate__
(pid=2836)     ray.actor.exit_actor()
(pid=2836)   File "/Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.8/site-packages/ray/actor.py", line 1091, in exit_actor
(pid=2836)     raise exit
(pid=2836) SystemExit: 0
(pid=2836)

------------------
## 15 min break :)
------------------


(while the above experiment is running (and hopefully learning))


## How do we extract any checkpoint from a trial of a tune.run?

In [47]:
# The previous tune.run (the one we did before the exercise) returned an Analysis object, from which we can access any checkpoint
# (given we set checkpoint_freq or checkpoint_at_end to reasonable values) like so:
print(analysis)
# Get all trials (we only have one).
trials = analysis.trials
# Assuming, the first trial was the best, we'd like to extract this trial's best checkpoint "":
best_checkpoint = analysis.get_best_checkpoint(trial=trials[0], metric="episode_reward_mean", mode="max")
print(f"Found best checkpoint for trial #2: {best_checkpoint}")

# Undo the grid-search config, which RLlib doesn't understand.
rllib_config = tune_config.copy()
rllib_config["lr"] = 0.00005
rllib_config["train_batch_size"] = 4000

# Restore a RLlib Trainer from the checkpoint.
new_trainer = PPOTrainer(config=rllib_config)
new_trainer.restore(best_checkpoint)
new_trainer

2021-06-24 11:14:28,753	WARNING ppo.py:135 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=5 num_envs_per_worker=5 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 160.


Found best checkpoint for trial #2: /Users/sven/ray_results/PPO/PPO_MultiAgentArena_62cb1_00000_0_2021-06-24_10-51-37/checkpoint_000200/checkpoint-200


(raylet) /Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   warnings.warn(
(raylet) /Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   warnings.warn(
(raylet) /Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update y

In [51]:
out = Output()
display.display(out)

with out:
    obs = env.reset()
    while True:
        a1 = new_trainer.compute_action(obs["agent1"], policy_id="policy1")
        a2 = new_trainer.compute_action(obs["agent2"], policy_id="policy2")
        actions = {"agent1": a1, "agent2": a2}
        obs, rewards, dones, _ = env.step(actions)

        out.clear_output(wait=True)
        env.render()
        time.sleep(0.07)

        if dones["agent1"] is True:
            break


Output()

## Let's talk about customization options

### Deep Dive: How do we customize RLlib's RL loop?

RLlib offers a callbacks API that allows you to add custom behavior to
all major events during the environment sampling- and learning process.

**Our problem:** So far, we can only see standard stats, such as rewards, episode lengths, etc..
This does not give us enough insights sometimes into important questions, such as: How many times
have both agents collided? or How many times has agent1 discovered a new field?

In the following cell, we will create custom callback "hooks" that will allow us to
add these stats to the returned metrics dict, and which will therefore be displayed in tensorboard!

For that we will override RLlib's DefaultCallbacks class and implement the
`on_episode_start`, `on_episode_step`, and `on_episode_end` methods therein:


In [52]:
# Override the DefaultCallbacks with your own and implement any methods (hooks)
# that you need.
from ray.rllib.agents.callbacks import DefaultCallbacks
from ray.rllib.evaluation.episode import MultiAgentEpisode


class MyCallbacks(DefaultCallbacks):
    def on_episode_start(self,
                         *,
                         worker,
                         base_env,
                         policies,
                         episode: MultiAgentEpisode,
                         env_index,
                         **kwargs):
        # We will use the `MultiAgentEpisode` object being passed into
        # all episode-related callbacks. It comes with a user_data property (dict),
        # which we can write arbitrary data into.

        # At the end of an episode, we'll transfer that data into the `hist_data`, and `custom_metrics`
        # properties to make sure our custom data is displayed in TensorBoard.

        # The episode is starting:
        # Set per-episode object to capture, which states (observations)
        # have been visited by agent1.
        episode.user_data["new_fields_discovered"] = 0
        # Set per-episode agent2-blocks counter (how many times has agent2 blocked agent1?).
        episode.user_data["num_collisions"] = 0

    def on_episode_step(self,
                        *,
                        worker,
                        base_env,
                        episode: MultiAgentEpisode,
                        env_index,
                        **kwargs):
        # Get both rewards.
        ag1_r = episode.prev_reward_for("agent1")
        ag2_r = episode.prev_reward_for("agent2")

        # Agent1 discovered a new field.
        if ag1_r == 1.0:
            episode.user_data["new_fields_discovered"] += 1
        # Collision.
        elif ag2_r == 1.0:
            episode.user_data["num_collisions"] += 1

    def on_episode_end(self,
                       *,
                       worker,
                       base_env,
                       policies,
                       episode: MultiAgentEpisode,
                       env_index,
                       **kwargs):
        # Episode is done:
        # Write scalar values (sum over rewards) to `custom_metrics` and
        # time-series data (rewards per time step) to `hist_data`.
        # Both will be visible then in TensorBoard.
        episode.custom_metrics["new_fields_discovered"] = episode.user_data["new_fields_discovered"]
        episode.custom_metrics["num_collisions"] = episode.user_data["num_collisions"]


In [53]:
# Setting up our config to point to our new custom callbacks class:
config = {
    "env": MultiAgentArena,
    "callbacks": MyCallbacks,  # by default, this would point to `rllib.agents.callbacks.DefaultCallbacks`, which does nothing.
    "num_workers": 5,  # we know now: this speeds up things!
}

tune.run(
    "PPO",
    config=config,
    stop={"training_iteration": 20},
    checkpoint_at_end=True,
    # If you'd like to restore the tune run from an existing checkpoint file, you can do the following:
    #restore="/Users/sven/ray_results/PPO/PPO_MultiAgentArena_fd451_00000_0_2021-05-25_15-13-26/checkpoint_000010/checkpoint-10",
)

Trial name,status,loc
PPO_MultiAgentArena_bdcf5_00000,PENDING,


(pid=3056) 2021-06-24 11:44:23,017	INFO trainer.py:671 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=3056) 2021-06-24 11:44:23,017	INFO trainer.py:696 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=3056) 2021-06-24 11:44:32,054	WARNING util.py:53 -- Install gputil for GPU system monitoring.


Result for PPO_MultiAgentArena_bdcf5_00000:
  agent_timesteps_total: 4000
  custom_metrics:
    new_fields_discovered_max: 42
    new_fields_discovered_mean: 33.85
    new_fields_discovered_min: 22
    num_collisions_max: 6
    num_collisions_mean: 0.9
    num_collisions_min: 0
  date: 2021-06-24_11-44-35
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 4.800000000000013
  episode_reward_mean: -8.309999999999997
  episode_reward_min: -27.000000000000043
  episodes_this_iter: 20
  episodes_total: 20
  experiment_id: ceeef6f183654ea083d83a3b87cf5a84
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 4.999999873689376e-05
          entropy: 1.3663787841796875
          entropy_coeff: 0.0
          kl: 0.02001993916928768
          model: {}
          policy_loss: -0.05442614480853081
          total_loss: 16.660539627075195
          vf_explaine

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_bdcf5_00000,RUNNING,192.168.0.179:3056,1,3.05959,4000,-8.31,4.8,-27,100


Result for PPO_MultiAgentArena_bdcf5_00000:
  agent_timesteps_total: 12000
  custom_metrics:
    new_fields_discovered_max: 51
    new_fields_discovered_mean: 37.5
    new_fields_discovered_min: 20
    num_collisions_max: 6
    num_collisions_mean: 1.0
    num_collisions_min: 0
  date: 2021-06-24_11-44-40
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 19.199999999999932
  episode_reward_mean: -2.6449999999999956
  episode_reward_min: -28.500000000000064
  episodes_this_iter: 20
  episodes_total: 60
  experiment_id: ceeef6f183654ea083d83a3b87cf5a84
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.30000001192092896
          cur_lr: 4.999999873689376e-05
          entropy: 1.3080261945724487
          entropy_coeff: 0.0
          kl: 0.01789451576769352
          model: {}
          policy_loss: -0.05154493451118469
          total_loss: 14.574112892150879
          vf_explai

...

Result for PPO_MultiAgentArena_bdcf5_00000:
  agent_timesteps_total: 76000
  custom_metrics:
    new_fields_discovered_max: 59
    new_fields_discovered_mean: 47.79
    new_fields_discovered_min: 32
    num_collisions_max: 34
    num_collisions_mean: 1.5
    num_collisions_min: 0
  date: 2021-06-24_11-45-26
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 29.69999999999989
  episode_reward_mean: 12.809999999999963
  episode_reward_min: -5.699999999999989
  episodes_this_iter: 20
  episodes_total: 380
  experiment_id: ceeef6f183654ea083d83a3b87cf5a84
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.44999998807907104
          cur_lr: 4.999999873689376e-05
          entropy: 0.8129876255989075
          entropy_coeff: 0.0
          kl: 0.017267907038331032
          model: {}
          policy_loss: -0.042705416679382324
          total_loss: 37.50997543334961
          vf_expla

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_bdcf5_00000,RUNNING,192.168.0.179:3056,19,53.6499,76000,12.81,29.7,-5.7,100


Result for PPO_MultiAgentArena_bdcf5_00000:
  agent_timesteps_total: 80000
  custom_metrics:
    new_fields_discovered_max: 59
    new_fields_discovered_mean: 47.89
    new_fields_discovered_min: 32
    num_collisions_max: 34
    num_collisions_mean: 1.93
    num_collisions_min: 0
  date: 2021-06-24_11-45-29
  done: true
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 29.69999999999989
  episode_reward_mean: 13.232999999999954
  episode_reward_min: -5.699999999999989
  episodes_this_iter: 20
  episodes_total: 400
  experiment_id: ceeef6f183654ea083d83a3b87cf5a84
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.44999998807907104
          cur_lr: 4.999999873689376e-05
          entropy: 0.8082447052001953
          entropy_coeff: 0.0
          kl: 0.018926914781332016
          model: {}
          policy_loss: -0.045885637402534485
          total_loss: 28.56772232055664
          vf_expla

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_bdcf5_00000,TERMINATED,,20,56.5668,80000,13.233,29.7,-5.7,100


2021-06-24 11:45:29,870	INFO tune.py:549 -- Total run time: 73.10 seconds (72.30 seconds for the tuning loop).


### Let's check tensorboard for the new custom metrics!

1. Head over to the Anyscale project view and click on the "TensorBoard" butten:

<img src="images/tensorboard_button.png" width=1000>

Alternatively - if you ran this locally on your own machine:

1. Head over to ~/ray_results/PPO/PPO_MultiAgentArena_[some key]_00000_0_[date]_[time]/
1. In that directory, you should see a `event.out....` file.
1. Run `tensorboard --logdir .` and head to https://localhost:6006

<img src="images/tensorboard.png" width=800>


### Deep Dive: Writing custom Models in tf or torch.

In [57]:
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.framework import try_import_tf, try_import_torch

tf1, tf, tf_version = try_import_tf()
torch, nn = try_import_torch()


# Custom Neural Network Models.
class MyKerasModel(TFModelV2):
    """Custom model for policy gradient algorithms."""

    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        """Build a simple [16, 16]-MLP (+ value branch)."""
        super(MyKerasModel, self).__init__(obs_space, action_space,
                                           num_outputs, model_config, name)
        
        # Keras Input layer.
        self.inputs = tf.keras.layers.Input(
            shape=obs_space.shape, name="observations")

        # Hidden layer (shared by action logits outputs and value output).
        layer_1 = tf.keras.layers.Dense(
            16,
            name="layer1",
            activation=tf.nn.relu)(self.inputs)
        
        # Action logits output.
        logits = tf.keras.layers.Dense(
            num_outputs,
            name="out",
            activation=None)(layer_1)

        # "Value"-branch (single node output).
        # Used by several RLlib algorithms (e.g. PPO) to calculate an observation's value.
        value_out = tf.keras.layers.Dense(
            1,
            name="value",
            activation=None)(layer_1)

        # The actual Keras model:
        self.base_model = tf.keras.Model(self.inputs,
                                         [logits, value_out])

    def forward(self, input_dict, state, seq_lens):
        """Custom-define your forard pass logic here."""
        # Pass inputs through our 2 layers and calculate the "value"
        # of the observation and store it for when `value_function` is called.
        logits, self.cur_value = self.base_model(input_dict["obs"])
        return logits, state

    def value_function(self):
        """Implement the value branch forward pass logic here:
        
        We will just return the already calculated `self.cur_value`.
        """
        assert self.cur_value is not None, "Must call `forward()` first!"
        return tf.reshape(self.cur_value, [-1])


class MyTorchModel(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        """Build a simple [16, 16]-MLP (+ value branch)."""
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs,
                              model_config, name)
        nn.Module.__init__(self)

        self.device = torch.device("cuda"
                                   if torch.cuda.is_available() else "cpu")

        # Hidden layer (shared by action logits outputs and value output).
        self.layer_1 = nn.Linear(obs_space.shape[0], 16).to(self.device)

        # Action logits output.
        self.layer_out = nn.Linear(16, num_outputs).to(self.device)

        # "Value"-branch (single node output).
        # Used by several RLlib algorithms (e.g. PPO) to calculate an observation's value.
        self.value_branch = nn.Linear(16, 1).to(self.device)
        self.cur_value = None

    def forward(self, input_dict, state, seq_lens):
        """Custom-define your forard pass logic here."""
        # Pass inputs through our 2 layers.
        layer_1_out = self.layer_1(input_dict["obs"])
        logits = self.layer_out(layer_1_out)

        # Calculate the "value" of the observation and store it for
        # when `value_function` is called.
        self.cur_value = self.value_branch(layer_1_out).squeeze(1)

        return logits, state

    def value_function(self):
        """Implement the value branch forward pass logic here:
        
        We will just return the already calculated `self.cur_value`.
        """
        assert self.cur_value is not None, "Must call `forward()` first!"
        return self.cur_value


In [58]:
# Do a quick test on the custom model classes.
test_model_tf = MyKerasModel(
    obs_space=gym.spaces.Box(-1.0, 1.0, (2, )),
    action_space=None,
    num_outputs=2,
    model_config={},
    name="MyModel",
)

print("TF-output={}".format(test_model_tf({"obs": np.array([[0.5, 0.5]])})))

# For PyTorch, you can do:
#test_model_torch = MyTorchModel(
#    obs_space=gym.spaces.Box(-1.0, 1.0, (2, )),
#    action_space=None,
#    num_outputs=2,
#    model_config={},
#    name="MyModel",
#)
#print("Torch-output={}".format(test_model_torch({"obs": torch.from_numpy(np.array([[0.5, 0.5]], dtype=np.float32))})))


TF-output=(<tf.Tensor 'model_8/out/BiasAdd:0' shape=(1, 2) dtype=float32>, [])


In [56]:
# Set up our custom model and re-run the experiment.
config.update({
    "model": {
        "custom_model": MyKerasModel,
        "custom_model_config": {
            #"layers": [128, 128],
        },
    },
})

tune.run(
    "PPO",
    config=config,
    stop={
        "training_iteration": 3,
    },
)


Trial name,status,loc
PPO_MultiAgentArena_e96ef_00000,PENDING,


(pid=3058) 2021-06-24 11:45:35,723	INFO trainer.py:671 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=3058) 2021-06-24 11:45:35,723	INFO trainer.py:696 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(raylet) /Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   warnings.warn(
(raylet) /Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   warnings.

Result for PPO_MultiAgentArena_e96ef_00000:
  agent_timesteps_total: 2400
  custom_metrics:
    new_fields_discovered_max: 40
    new_fields_discovered_mean: 29.583333333333332
    new_fields_discovered_min: 15
    num_collisions_max: 7
    num_collisions_mean: 0.75
    num_collisions_min: 0
  date: 2021-06-24_11-45-46
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 1.5000000000000093
  episode_reward_mean: -14.800000000000002
  episode_reward_min: -37.50000000000004
  episodes_this_iter: 12
  episodes_total: 12
  experiment_id: f92dc745139c4b73a2c52cbb5ede0c35
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 0.0005000000237487257
          entropy: 1.3625845909118652
          entropy_coeff: 0.0
          kl: 0.02165747620165348
          model: {}
          policy_loss: -0.021877726539969444
          total_loss: 22.76353645324707
      

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_e96ef_00000,RUNNING,192.168.0.179:3058,1,1.46996,2400,-14.8,1.5,-37.5,100


Result for PPO_MultiAgentArena_e96ef_00000:
  agent_timesteps_total: 14400
  custom_metrics:
    new_fields_discovered_max: 51
    new_fields_discovered_mean: 35.833333333333336
    new_fields_discovered_min: 15
    num_collisions_max: 15
    num_collisions_mean: 2.0277777777777777
    num_collisions_min: 0
  date: 2021-06-24_11-45-52
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 17.099999999999977
  episode_reward_mean: -4.604166666666655
  episode_reward_min: -37.50000000000004
  episodes_this_iter: 12
  episodes_total: 72
  experiment_id: f92dc745139c4b73a2c52cbb5ede0c35
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.15000000596046448
          cur_lr: 0.0005000000237487257
          entropy: 1.3478621244430542
          entropy_coeff: 0.0
          kl: 0.009249470196664333
          model: {}
          policy_loss: -0.014932680875062943
          total_loss: 16.79816

...

Result for PPO_MultiAgentArena_e96ef_00000:
  agent_timesteps_total: 168000
  custom_metrics:
    new_fields_discovered_max: 58
    new_fields_discovered_mean: 42.41
    new_fields_discovered_min: 31
    num_collisions_max: 7
    num_collisions_mean: 1.88
    num_collisions_min: 0
  date: 2021-06-24_11-47-12
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 28.49999999999994
  episode_reward_mean: 5.346000000000003
  episode_reward_min: -11.999999999999975
  episodes_this_iter: 12
  episodes_total: 840
  experiment_id: f92dc745139c4b73a2c52cbb5ede0c35
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.5062500238418579
          cur_lr: 0.0005000000237487257
          entropy: 1.2910246849060059
          entropy_coeff: 0.0
          kl: 0.013177911750972271
          model: {}
          policy_loss: -0.011083748191595078
          total_loss: 9.791471481323242
          vf_expla

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_e96ef_00000,RUNNING,192.168.0.179:3058,70,85.9171,168000,5.346,28.5,-12,100


2021-06-24 11:47:14,561	WARNING tune.py:506 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_e96ef_00000,RUNNING,192.168.0.179:3058,72,88.3456,172800,6.012,28.5,-14.1,100


(pid=3058) 2021-06-24 11:47:14,610	ERROR worker.py:409 -- SystemExit was raised from the worker
(pid=3058) Traceback (most recent call last):
(pid=3058)   File "python/ray/_raylet.pyx", line 595, in ray._raylet.task_execution_handler
(pid=3058)   File "python/ray/_raylet.pyx", line 453, in ray._raylet.execute_task
(pid=3058)   File "python/ray/_raylet.pyx", line 490, in ray._raylet.execute_task
(pid=3058)   File "python/ray/_raylet.pyx", line 497, in ray._raylet.execute_task
(pid=3058)   File "python/ray/_raylet.pyx", line 501, in ray._raylet.execute_task
(pid=3058)   File "python/ray/_raylet.pyx", line 451, in ray._raylet.execute_task.function_executor
(pid=3058)   File "/Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.8/site-packages/ray/_private/function_manager.py", line 563, in actor_method_executor
(pid=3058)     return method(__ray_actor, *args, **kwargs)
(pid=3058)   File "/Users/sven/opt/anaconda3/envs/rllib_tutorial/lib/python3.8/site-packages/ray/tune/trainable.py",

### Deep Dive: A closer look at RLlib's APIs and components
#### (Depending on time left and amount of questions having been accumulated :)

We already took a quick look inside an RLlib Trainer object and extracted its Policy(ies) and the Policy's model (neural network). Here is a much more detailed overview of what's inside a Trainer object.

At the core is the so-called `WorkerSet` sitting under `Trainer.workers`. A WorkerSet is a group of `RolloutWorker` (`rllib.evaluation.rollout_worker.py`) objects that always consists of a "local worker" (`Trainer.workers.local_worker()`) and n "remote workers" (`Trainer.workers.remote_workers()`).



<img src="images/rllib_structure.png" width=1000>

### Scaling RLlib

Scaling RLlib works by parallelizing the "jobs" that the remote `RolloutWorkers` do. In a vanilla RL algorithm, like PPO, DQN, and many others, the `@ray.remote` labeled RolloutWorkers in the figure above are responsible for interacting with one or more environments and thereby collecting experiences. Observations are produced by the environment, actions are then computed by the Policy(ies) copy located on the remote worker and sent to the environment in order to produce yet another observation. This cycle is repeated endlessly and only sometimes interrupted to send experience batches ("train batches") of a certain size to the "local worker". There these batches are used to call `Policy.learn_on_batch()`, which performs a loss calculation, followed by a model weights update, and a subsequent weights broadcast back to all the remote workers.



## Time for Q&A

...

## Thank you for listening and participating!

### Here are a couple of links that you may find useful.

- The <a href="https://github.com/sven1977/rllib_tutorials.git">github repo of this tutorial</a>.
- <a href="https://docs.ray.io/en/master/rllib.html">RLlib's documentation main page</a>.
- <a href="http://discuss.ray.io">Our discourse forum</a> to ask questions on RLlib.
- Our <a href="https://forms.gle/9TSdDYUgxYs8SA9e8">Slack channel</a> for interacting with other Ray RLlib users.
- The <a href="https://github.com/ray-project/ray/blob/master/rllib/examples/">RLlib examples scripts folder</a> with tons of examples on how to do different stuff with RLlib.
- A <a href="https://medium.com/distributed-computing-with-ray/reinforcement-learning-with-rllib-in-the-unity-game-engine-1a98080a7c0d">blog post on training with RLlib inside a Unity3D environment</a>.
